# Frequency-content of LFP signals

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from elephant.spike_train_generation import inhomogeneous_poisson_process
from quantities import Hz, ms
import neo
import neuron
import LFPy
from brainsignals.plotting_convention import mark_subplots, simplify_axes
from brainsignals.neural_simulations import return_hay_cell
import brainsignals.neural_simulations as ns

ns.load_mechs_from_folder(ns.cell_models_folder)
np.random.seed(1534)

In [ ]:

num_elecs = 1
# Define electrode parameters
elec_params = {
    'sigma': 0.3,      # extracellular conductivity
    'x': np.array([10]),  # electrode positions
    'y': np.zeros(num_elecs),
    'z': np.zeros(num_elecs),
    'method': 'pointsource'
}

def insert_synapses(cell, synapse_params, syn_idxs,  spiketimes):
    for num, idx in enumerate(syn_idxs):
        synapse_params.update({'idx' : idx})
        s = LFPy.Synapse(cell, **synapse_params)
        s.set_spike_times(np.array([spiketimes[num]]))

In [ ]:
tstop = 8000
avrg_rate = 2000 
num_syns = int(tstop / 1000 * avrg_rate)

dt = 2**-4
num_tsteps = int(tstop / dt + 1)
tvec = np.arange(num_tsteps) * dt

cell = return_hay_cell(tstop=tstop, dt=dt, make_passive=True)
syn_idxs = cell.get_rand_idx_area_norm(section="allsec", nidx=num_syns,
                                   z_min=-300, z_max=100)

cell.__del__()
spiketime_dict = {}
signal_clrs = {}

t_stop = tstop * ms

freq_3 = 2  # Hz

# Create the rate functions of the synaptic input. They must non-negative, 
# and should always have some input
rate_signal_0 = np.zeros(len(tvec)) + 1
rate_signal_3 = np.sign(np.sin(2 * np.pi * freq_3 * tvec / 1000)) + 1.2

rate_0 = neo.AnalogSignal(np.array(rate_signal_0) * avrg_rate * Hz,
                      sampling_rate=(1/dt * 1000)*Hz, t_stop=t_stop)
rate_3 = neo.AnalogSignal(np.array(rate_signal_3)  * avrg_rate * Hz,
                      sampling_rate=(1/dt * 1000)*Hz, t_stop=t_stop)

rate_dict = {"stationary_delta": rate_0,
             "stationary_exp": rate_0,
            "up_down_states": rate_3,
            "up_down_states_delta": rate_3,
            }
tau_delta = dt
tau = 3  # ms
tau_dict = {"stationary_delta": tau_delta,
            "stationary_exp": tau,
            "up_down_states": tau,
            "up_down_states_delta": tau_delta,}
weigth = 0.025
weight_dict = {"stationary_delta": weigth * tau / dt,  # Delta pulses gives much weaker signals, scale up for visual reasons
               "stationary_exp": weigth,
               "up_down_states": weigth,
               "up_down_states_delta": weigth * tau / dt}



synapse_params = {
    'syntype' : 'ExpSynI',      #conductance based exponential synapse
    'record_current' : False,    #record synaptic currents
}

# To ensure the same number of spikes, we deliberately overshoot, and then extract the
# wanted number of spikes
st_0 = inhomogeneous_poisson_process(rate_0 * 1.2)
st_3 = inhomogeneous_poisson_process(rate_3 * 1.2)

st_0 = np.random.choice(st_0, size=num_syns, replace=False) * 1000
st_3 = np.random.choice(st_3, size=num_syns, replace=False) * 1000

sim_order = ["stationary_delta", "stationary_exp", "up_down_states_delta", "up_down_states"]

spiketime_dict = {"stationary_delta": np.array(st_0),
                  "stationary_exp": np.array(st_0),
                  "up_down_states": st_3,
                  "up_down_states_delta": st_3,
                 }

# Note: These parameters are specifically chosen to make "clean" PSD curves, and 
# because of the substantial averaging, the stereotypical "ringing" effect of taking the 
# PSD of a square pulse is removed

divide_into_welch = 16
welch_dict = {'Fs': 1000 / dt,
              'NFFT': int(num_tsteps/divide_into_welch),
              'noverlap': int(num_tsteps/divide_into_welch/2.),
              'detrend': 'mean',
              'scale_by_freq': True,
              }

fr_psd_dict = {}
lfp_dict = {}
psd_dict = {}

for i, (name, spiketimes) in enumerate(spiketime_dict.items()):
    print(i, name)

    cell = return_hay_cell(tstop=tstop, dt=dt, make_passive=True)

    synapse_params["tau"] = tau_dict[name]
    synapse_params["weight"] = weight_dict[name]
    insert_synapses(cell, synapse_params, syn_idxs, spiketimes)
    
    cell.simulate(rec_imem=True, rec_vmem=True)

    electrode = LFPy.RecExtElectrode(cell, **elec_params)
    lfp = electrode.get_transformation_matrix() @ cell.imem * 1000

    cell.__del__()
    lfp_dict[name] = lfp    
    freq, psd = ns.return_freq_and_psd_welch(lfp_dict[name], welch_dict)
    psd_dict[name] = [freq, psd]


In [ ]:
legend_dict = {"stationary_delta": "stationary\ndelta synapses",
               "stationary_exp": "stationary\nexponential synapses",
               "up_down_states": "up-and-down states\nexponential synapses",
               "up_down_states_delta": "up-and-down states\ndelta synapses",
              }

signal_clrs = {"stationary_delta": 'k',
               "stationary_exp": 'gray',
               "up_down_states": 'r',
               "up_down_states_delta": 'orange',
              }
signal_lws = {"stationary_delta": 4,
               "stationary_exp": 2,
               "up_down_states": 1.0,
               "up_down_states_delta": 2,
              }

plt.close("all")
fig = plt.figure(figsize=[6, 3])

ax_morph = fig.add_axes([0.00, 0.01, 0.15, 0.98], aspect=1, xticks=[],
                        yticks=[], frameon=False)
ax_psd = fig.add_axes([0.68, 0.13, 0.27, 0.75], xlabel="frequency (Hz)", ylabel="µV²/Hz",
                      xlim=[1, 5000], ylim=[1e-8, 1e-1],
                      title="LFP-PSD")
cell = return_hay_cell(tstop=tstop, dt=dt, make_passive=True)
ax_morph.plot(cell.x.T, cell.z.T, c='k', lw=1, zorder=-1)

ax_morph.scatter(cell.x.mean(axis=1)[syn_idxs], cell.z.mean(axis=1)[syn_idxs],
              c='b', s=1)
ax_morph.plot(elec_params["x"], elec_params["z"], 'co')

lines = []
line_names = []
for i, name in enumerate(sim_order):

    xlim = [tstop - 1000, tstop]
    tbar = [tstop - 100, tstop]
    bartext = "100 ms"
    ax_fr = fig.add_axes([0.21, 0.8 - i / len(spiketime_dict) * 0.95, 0.32, 0.1],
                         xlim=xlim, ylim=[-0.1, 1.2], frameon=False, yticks=[], xticks=[])
    ax_fr.set_title(legend_dict[name], pad=-2)

    ax_fr.plot(tbar, [-0.1, -0.1], c='k', lw=2, clip_on=False, zorder=1e5)
    ax_fr.text(np.mean(tbar), -0.17, bartext, va='top', ha='center')

    ax_fr.plot(tvec, (lfp_dict[name][0] - np.mean(lfp_dict[name][0])) / np.max(np.abs(lfp_dict[name])) + 0.5, 
           c=signal_clrs[name])    
    l, = ax_fr.plot(tvec, rate_dict[name] / np.max(rate_dict["up_down_states"]), c='0.7', ls='--')
    ax_psd.loglog(psd_dict[name][0][1:], psd_dict[name][1][0][1:], 
                  c=signal_clrs[name], lw=signal_lws[name])
    
ax_psd.loglog([500, 5000], [2e-3, 2e-4], c='k', lw=2)
ax_psd.text(5500, 2e-4, "1/$f$", c='k')

ax_psd.loglog([500, 5000], [2e-5, 2e-7], c='k', lw=2)
ax_psd.text(5500, 2e-7, "1/$f^2$", c='k')

fig.legend([l], ["synaptic input profile"], frameon=False, ncol=2, loc=(0.23, 0.001))

mark_subplots(ax_morph, "A", xpos=0.1, ypos=0.97)
mark_subplots(fig.axes[2:], "BCDE")
mark_subplots(ax_psd, "F", ypos=1.08)
simplify_axes(fig.axes)
ax_psd.set_xticks([1, 10, 100, 1000])
ax_psd.grid(True)

fig.savefig("fig_LFP_powerlaws_input_dynamics.pdf")